In [128]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# Fastnade med att försöka extrahera adress, se kommentar #1.

In [210]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')

vasastan_url = "https://www.hemnet.se/salda/bostader?location_ids%5B%5D=925970"

driver = webdriver.Chrome("chromedriver.exe", chrome_options=options)
driver.get(vasastan_url)
time.sleep(2)
try:
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,'//*[@class="hcl-button hcl-button--primary"]'))).click()

except(org.openqa.selenium.StaleElementReferenceException):
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,'//*[@class="hcl-button hcl-button--primary"]'))).click()



time.sleep(2)
elements = driver.find_elements_by_class_name("sold-property-link")
test_dict = {}

for i in range(0,len(elements)):
    elements[i].click()
    time.sleep(2)
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(2)
    #1. adress = driver.find_element_by_xpath("//*[@class='hcl-heading hcl-heading--size1')")
    #print(adress.text)
    
    price = driver.find_element_by_class_name("sold-property__price-value")
    price = price.text

    
    attributes = driver.find_element_by_class_name("sold-property__attributes")
    attributes_text = attributes.text
    stats = driver.find_element_by_class_name("sold-property__price-stats")
    stats_text = stats.text
    
    attributes_stats_text = attributes_text+ "\n" + stats_text
    test_dict[i] = attribute_extractor(attributes_stats_text)
    time.sleep(2)
    driver.close();
    time.sleep(2)
    driver.switch_to.window(driver.window_handles[0]);
    time.sleep(2)
    
    print(test_dict[i])



C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  
C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  
C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


InvalidSelectorException: Message: invalid selector: Unable to locate an element with the xpath expression //*[@class='hcl-heading hcl-heading--size1') because of the following error:
SyntaxError: Failed to execute 'evaluate' on 'Document': The string '//*[@class='hcl-heading hcl-heading--size1')' is not a valid XPath expression.
  (Session info: headless chrome=97.0.4692.71)
Stacktrace:
Backtrace:
	Ordinal0 [0x00766903+2517251]
	Ordinal0 [0x006FF8E1+2095329]
	Ordinal0 [0x00602848+1058888]
	Ordinal0 [0x00604F44+1068868]
	Ordinal0 [0x00604E0E+1068558]
	Ordinal0 [0x00605070+1069168]
	Ordinal0 [0x0062D1C2+1233346]
	Ordinal0 [0x0062D63B+1234491]
	Ordinal0 [0x00657812+1406994]
	Ordinal0 [0x0064650A+1336586]
	Ordinal0 [0x00655BBF+1399743]
	Ordinal0 [0x0064639B+1336219]
	Ordinal0 [0x006227A7+1189799]
	Ordinal0 [0x00623609+1193481]
	GetHandleVerifier [0x008F5904+1577972]
	GetHandleVerifier [0x009A0B97+2279047]
	GetHandleVerifier [0x007F6D09+534521]
	GetHandleVerifier [0x007F5DB9+530601]
	Ordin

In [203]:
def attribute_extractor(text):
    
    #text = text_element_list.text
    word_list = []
    while text.find("\n") != -1:
        cut_off = text.find("\n")
        word_list.append(text[0:cut_off])
        text = text[cut_off+1:len(text)]   
    word_list.append(text)

    attribute_list = []
    attribute_values = []


    for i in range(0, len(word_list)):
        if i % 2 == 0:
            attribute_list.append(word_list[i])
        else:
            attribute_values.append(word_list[i])

    res = dict(zip(attribute_list, attribute_values))
    return res

In [178]:
yes = attribute_extractor(attributes)
yes

{'Bostadstyp': 'Lägenhet',
 'Upplåtelseform': 'Bostadsrätt',
 'Antal rum': '2 rum',
 'Boarea': '38 m²',
 'Balkong': 'Nej',
 'Våning': '1 av 5, hiss finns',
 'Byggår': '1928',
 'Avgift/månad': '2 147 kr/mån',
 'Driftskostnad': '5 250 kr/år'}